
# Assignment #2

Pedro Stramantinoli P. Cagliume Gomes 175955

Ruy Castilho Barrichelo 177012

## Importação dos Dados

In [1]:
import pandas as pd
import os

def load_fashionMNIST_train():
    csv_path = os.path.join("Data/fashion-mnist-dataset", "fashion-mnist_train.csv")
    return pd.read_csv(csv_path)

def load_fashionMNIST_tests():
    csv_path = os.path.join("Data/fashion-mnist-dataset", "fashion-mnist_test.csv")
    return pd.read_csv(csv_path)

## Feature Scalling e Normalização

In [2]:
import numpy as np

def normalize(train, test):
    train_mean = np.mean(train, axis=0)
    train = train - train_mean
    test = test - train_mean
    
    return train, test

def scale(X):
    max_array = np.max(X, axis=0)
    X = X / max_array[None, :]
    return X

## Geração de matrizes de Features e Target

### Setup inicial dos dados

In [3]:
import numpy as np

#carrega os dados
data_train = load_fashionMNIST_train()
data_test = load_fashionMNIST_tests()

# Separação em Features e Target

data_train_target = np.array(data_train["label"])
data_test_target = np.array(data_test["label"])

data_train = np.array(data_train)
data_test = np.array(data_test)

data_train = np.delete(data_train, 0, axis=1)
data_test = np.delete(data_test, 0, axis=1)

# Normalização
normalized_data_train, normalized_data_test = normalize(data_train,data_test)

# Scaling
scaled_data_train = scale(normalized_data_train)
scaled_data_test = scale(normalized_data_test)

# Adicionando coluna de 1's
ready_data_train = np.c_[np.ones((len(scaled_data_train),1)), scaled_data_train]
ready_data_test = np.c_[np.ones((len(scaled_data_test),1)), scaled_data_test]

def getTrainSet():
    return ready_data_train, data_train_target

def getTestSet():
    return ready_data_test,data_test_target

## Gerador de conjuntos de Cross Validation

In [4]:
# Cross Validation Generation 

from sklearn.model_selection import train_test_split, KFold

# Retorna um gerador de indices
def generate_sets(TRAINING_DATA,type='kfold'):

    # Cross validation using train_test_split
    if (type == 'split'):
       return train_test_split(TRAINING_DATA,test_size=0.2,random_state=0)

    # Cross validation using K-Fold
    # K = 5, Shuffle = true, Seed = 21
    elif (type == 'kfold'):
        kfold_seed = 21

        kfold = KFold(n_splits=5, shuffle=True, random_state=kfold_seed)
        return kfold.split(TRAINING_DATA)

### Sigmoid and Loss functions

In [5]:
import numpy as np

def sigmoid(z):
    return 1.0 / (1+np.exp(-z))

def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

## Softmax

In [83]:
import numpy as np

def softmax(z):
    expZ = np.exp(z - np.max(z))
    return expZ / np.sum(expZ, axis=1, keepdims=True)


## Success/Error Percentages

In [6]:
import numpy as np

def error_rate(prediction, y):
    return np.mean( prediction != y )

def success_rate(prediction, y):
    return np.mean( prediction == y )

##  Binary Logistic Regression

In [7]:
import numpy as np


def binary_logistic_regression(x, y, learningRate, numIterations):
    xTrans = x.transpose()
    theta = np.zeros((x.shape[1],1))
    y = y.reshape((y.shape[0],1))
    m = x.shape[0]

    for i in range(numIterations):
        z = np.dot(x, theta)
        hypothesis = sigmoid(z)
        
        gradient = np.dot(xTrans, (hypothesis - y)) / m
        theta -= learningRate * gradient
        
    return theta.reshape(x.shape[1])
    

## Set the target according to the label

In [13]:
def setTarget(target,label):
    return (target == label)

### Perform Logistic Regression for each class (Binary Classifier)

In [14]:
import numpy as np

successForEachTheta = np.ndarray(5)
errorForEachTheta = np.ndarray(5)

X_Train,Target_Train = getTrainSet()

targetForEachClass = transformTarget(Target_Train)
thetaForEachClass = np.ndarray((5, X_Train.shape[1], 10))

indices_generator = generate_sets(X_Train)

layerIndex = 0
for train_index, val_index in indices_generator:
    # data for training and validation
    x_train = X_Train[train_index]
    y_train = Target_Train[train_index]
    x_val = X_Train[val_index]
    y_val = Target_Train[val_index]
    
    # calculate thetas for each class
    for i in range(10):
        transformedTarget = setTarget(y_train,i)
        thetaForEachClass[layerIndex,:,i] = binary_logistic_regression(x_train, transformedTarget, 0.01, 500)
    
    # calculate the sigmoid
    z = x_val.dot(thetaForEachClass[layerIndex,:,:])
    h = sigmoid(z)
    
    # make de prediction and calculate both success and error for each Theta
    y_predict = np.argmax(h,axis=1)
    errorForEachTheta[layerIndex] = error_rate(y_predict,y_val)
    successForEachTheta[layerIndex] = success_rate(y_predict,y_val)
    layerIndex += 1

### Perform softmax

In [81]:
successSoftmax = np.zeros(5)
errorSoftmax = np.zeros(5)
indices_generator = generate_sets(X_Train)
layerIndex = 0

for train_index, val_index in indices_generator:
    # data for training and validation
    x_train = X_Train[train_index]
    y_train = Target_Train[train_index]
    x_val = X_Train[val_index]
    y_val = Target_Train[val_index]
    
    # calculate the softmax
    z = x_val.dot(thetaForEachClass[layerIndex,:,:])
    hypothesis = softmax(z)
    
    # make de prediction and calculate both success and error for each Theta
    softMax_predict = np.argmax(hypothesis,axis=1)
    errorSoftmax[layerIndex] = error_rate(softMax_predict,y_val)
    successSoftmax[layerIndex] = success_rate(softMax_predict,y_val)
    layerIndex += 1

In [80]:
for i in range(5):
    print('theta ' + str(i))
    print('success: ' + str(successForEachTheta[i]))
    print('softmax: ' + str(successSoftmax[i]))
    print('-------')

theta 0
success: 0.7618333333333334
softmax: 0.7618333333333334
-------
theta 1
success: 0.7628333333333334
softmax: 0.7628333333333334
-------
theta 2
success: 0.76275
softmax: 0.76275
-------
theta 3
success: 0.7595
softmax: 0.7595
-------
theta 4
success: 0.7678333333333334
softmax: 0.7678333333333334
-------
